# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Apr 24 2023 11:26AM,261637,10,20230424-FL,SugarBear,Released
1,Apr 24 2023 11:10AM,261634,15,9412358,"Brookfield Pharmaceuticals, LLC",Released
2,Apr 24 2023 10:57AM,261633,15,9412356,Lonza,Released
3,Apr 24 2023 10:54AM,261631,10,MSP221053,"Methapharm, Inc.",Released
4,Apr 24 2023 10:54AM,261631,10,MSP221055,"Methapharm, Inc.",Released
5,Apr 24 2023 10:54AM,261631,10,MSP221056,"Methapharm, Inc.",Released
6,Apr 24 2023 10:54AM,261631,10,MSP221057,"Methapharm, Inc.",Released
7,Apr 24 2023 10:54AM,261631,10,MSP221058,"Methapharm, Inc.",Released
8,Apr 24 2023 10:50AM,261630,10,MSP221051,"Methapharm, Inc.",Released
9,Apr 24 2023 10:46AM,261629,15,30015914,"Alliance Pharma, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
14,261630,Released,1
15,261631,Released,5
16,261633,Released,1
17,261634,Released,1
18,261637,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
261630,NaN,1.0
261631,NaN,5.0
261633,NaN,1.0
261634,NaN,1.0
261637,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
261602,0.0,1.0
261612,0.0,29.0
261618,17.0,11.0
261619,0.0,1.0
261620,0.0,2.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
261602,0,1
261612,0,29
261618,17,11
261619,0,1
261620,0,2


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,261602,0,1
1,261612,0,29
2,261618,17,11
3,261619,0,1
4,261620,0,2


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,261602,,1
1,261612,,29
2,261618,17,11
3,261619,,1
4,261620,,2


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released      int32
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Apr 24 2023 11:26AM,261637,10,SugarBear
1,Apr 24 2023 11:10AM,261634,15,"Brookfield Pharmaceuticals, LLC"
2,Apr 24 2023 10:57AM,261633,15,Lonza
3,Apr 24 2023 10:54AM,261631,10,"Methapharm, Inc."
8,Apr 24 2023 10:50AM,261630,10,"Methapharm, Inc."
9,Apr 24 2023 10:46AM,261629,15,"Alliance Pharma, Inc."
31,Apr 24 2023 10:43AM,261627,10,Bio-PRF
49,Apr 24 2023 10:41AM,261628,10,Methapharm-G
50,Apr 24 2023 10:32AM,261626,10,"ClearSpec, LLC"
52,Apr 24 2023 10:26AM,261625,10,"CLINUVEL, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Apr 24 2023 11:26AM,261637,10,SugarBear,,1
1,Apr 24 2023 11:10AM,261634,15,"Brookfield Pharmaceuticals, LLC",,1
2,Apr 24 2023 10:57AM,261633,15,Lonza,,1
3,Apr 24 2023 10:54AM,261631,10,"Methapharm, Inc.",,5
4,Apr 24 2023 10:50AM,261630,10,"Methapharm, Inc.",,1
5,Apr 24 2023 10:46AM,261629,15,"Alliance Pharma, Inc.",,22
6,Apr 24 2023 10:43AM,261627,10,Bio-PRF,4,14
7,Apr 24 2023 10:41AM,261628,10,Methapharm-G,,1
8,Apr 24 2023 10:32AM,261626,10,"ClearSpec, LLC",,2
9,Apr 24 2023 10:26AM,261625,10,"CLINUVEL, Inc.",,2


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Apr 24 2023 11:26AM,261637,10,SugarBear,1,
1,Apr 24 2023 11:10AM,261634,15,"Brookfield Pharmaceuticals, LLC",1,
2,Apr 24 2023 10:57AM,261633,15,Lonza,1,
3,Apr 24 2023 10:54AM,261631,10,"Methapharm, Inc.",5,
4,Apr 24 2023 10:50AM,261630,10,"Methapharm, Inc.",1,
5,Apr 24 2023 10:46AM,261629,15,"Alliance Pharma, Inc.",22,
6,Apr 24 2023 10:43AM,261627,10,Bio-PRF,14,4
7,Apr 24 2023 10:41AM,261628,10,Methapharm-G,1,
8,Apr 24 2023 10:32AM,261626,10,"ClearSpec, LLC",2,
9,Apr 24 2023 10:26AM,261625,10,"CLINUVEL, Inc.",2,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Apr 24 2023 11:26AM,261637,10,SugarBear,1,
1,Apr 24 2023 11:10AM,261634,15,"Brookfield Pharmaceuticals, LLC",1,
2,Apr 24 2023 10:57AM,261633,15,Lonza,1,
3,Apr 24 2023 10:54AM,261631,10,"Methapharm, Inc.",5,
4,Apr 24 2023 10:50AM,261630,10,"Methapharm, Inc.",1,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Apr 24 2023 11:26AM,261637,10,SugarBear,1,NaN
1,Apr 24 2023 11:10AM,261634,15,"Brookfield Pharmaceuticals, LLC",1,NaN
2,Apr 24 2023 10:57AM,261633,15,Lonza,1,NaN
3,Apr 24 2023 10:54AM,261631,10,"Methapharm, Inc.",5,NaN
4,Apr 24 2023 10:50AM,261630,10,"Methapharm, Inc.",1,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Apr 24 2023 11:26AM,261637,10,SugarBear,1,0.0
1,Apr 24 2023 11:10AM,261634,15,"Brookfield Pharmaceuticals, LLC",1,0.0
2,Apr 24 2023 10:57AM,261633,15,Lonza,1,0.0
3,Apr 24 2023 10:54AM,261631,10,"Methapharm, Inc.",5,0.0
4,Apr 24 2023 10:50AM,261630,10,"Methapharm, Inc.",1,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,2093016,55,4.0
15,1308134,37,17.0
16,261602,1,0.0
18,261619,1,0.0
20,523243,51,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,2093016,55,4.0
1,15,1308134,37,17.0
2,16,261602,1,0.0
3,18,261619,1,0.0
4,20,523243,51,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,55,4.0
1,15,37,17.0
2,16,1,0.0
3,18,1,0.0
4,20,51,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,55.0
1,15,Released,37.0
2,16,Released,1.0
3,18,Released,1.0
4,20,Released,51.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,16,18,20
Status,,,,,
Executing,4.0,17.0,0.0,0.0,0.0
Released,55.0,37.0,1.0,1.0,51.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,16,18,20
0,Executing,4.0,17.0,0.0,0.0,0.0
1,Released,55.0,37.0,1.0,1.0,51.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,16,18,20
0,Executing,4.0,17.0,0.0,0.0,0.0
1,Released,55.0,37.0,1.0,1.0,51.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()